# Import library

In [ ]:
from requests_html import HTMLSession
import requests
import os
import argparse
import sys
import json
import time

In [ ]:
keywords = {
    'viettel': ['cua hang viettel', 'viettel store'],
    'fpt': ['fpt software', 'fpt'],
    'fujinet': ['fujinet', 'fujinet logo'],
    'vng': ['vng', 'vng logo'],
    'kms': ['kms', 'kms logo'],
}

In [ ]:
max_image_per_query = 100
image_dir_root = './download'
if not os.path.exists(image_dir_root):
    os.mkdir(image_dir_root)

In [ ]:
session = HTMLSession()

In [ ]:
def fetch_image_to_dir(query, output_dir, start_index=0, verbose=False):
    query= query.split()
    query='+'.join(query)

    url="https://www.google.co.in/search?q="+query+"&source=lnms&tbm=isch&tbs=sur:fc"
    res = session.get(url)

    while not(res.ok):
        time.sleep(1)
        res = session.get(url)

    i = start_index
    for value in res.html.find('.rg_meta'):
        json_data = json.loads(value.text)
        img = json_data['ou']
        ext = json_data['ity']

        if ext == '':
            continue
    
        if verbose:
            print(f'Downloading i={i}: {img}')

        try:
            raw_img = requests.get(img, timeout=10).content
            f = open(os.path.join(output_dir, f'img_{i}.{ext}'), 'wb')
            f.write(raw_img)
            f.close()
        except Exception as e:
            print(e)

        i += 1
        if (i - start_index) == max_image_per_query:
            break
    return i

In [ ]:
for company, queries in keywords.items():
    img_dir = os.path.join(image_dir_root, company)
    if not os.path.exists(img_dir):
        os.mkdir(img_dir)
        
    index = 0
    for query in queries:
        index = fetch_image_to_dir(query, img_dir, start_index=index, verbose=True)

# View some results